In [1]:
!pip install --quiet -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
# Load job titles data 
titles_df = pd.read_csv("/kaggle/input/df-dropna-lightcast-titles/lightcast-titles.csv")
titles = titles_df["Title"].tolist()

candidates_df = pd.read_excel('/kaggle/input/df-dropna-lightcast-titles/df-dropna.xlsx')

# Assuming 'df_candidates' and 'df_titles' are your DataFrames
# Encode candidate profiles, designation
candidate_profiles = candidates_df["candidate_profiles"].tolist()
jd_designations = candidates_df["designation"].tolist()

candidate_embeds = model.encode(candidate_profiles,show_progress_bar=False)
designation_embeds = model.encode(jd_designations,show_progress_bar=False)

# Encode titles dataset  
title_embeds = model.encode(titles, show_progress_bar=False)

In [4]:
# Cosine Similarity
candidate_cosine_sim = cosine_similarity(candidate_embeds, title_embeds)
jd_cosine_sim = cosine_similarity(designation_embeds,title_embeds)

# Get the index of the closest matches
cosine_best_match_candidate = candidate_cosine_sim.argmax(axis=1)
cosine_best_match_designation = jd_cosine_sim.argmax(axis=1)

In [5]:
# Get best matched titles
matched_titles_candidates = [titles[idx] for idx in cosine_best_match_candidate]
matched_titles_designation = [titles[idx] for idx in cosine_best_match_designation]


# Add to df
candidates_df['norm_profile'] = matched_titles_candidates
candidates_df['norm_designation'] = matched_titles_designation

In [6]:
# Reorder columns
columns_reordered = ['status', 'experience', 'present_ctc', 'expected_ctc', 'notice_period',
                     'candidate_profiles','norm_profile', 'job_categories', 
                     'designation', 'norm_designation','Relevant', 'exp_min', 'exp_max', 'ctc_fixed_min', 
                     'ctc_fixed_max']

# Reindex the DataFrame with the reordered columns
candidates_df = candidates_df.reindex(columns=columns_reordered)

candidates_df.head()  # Displaying the reordered DataFrame

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
0,ACCEPTED,2.00,3.6,4.14,15,Senior Sales Executive,Sales Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
1,ACCEPTED,11.00,3.0,4.00,1,Marketing executive,Marketing Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
2,ACCEPTED,2.50,5.0,5.75,15,Android Developer,Android Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
3,ACCEPTED,4.08,7.0,8.05,15,Android Mobile Application Developer|||Android...,React Native Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
4,ACCEPTED,3.50,4.0,8.00,1,Devops engineer,DevOps Engineer,Developer Operations,Developer operations,Operations Developer,1,1.0,5.0,6.0,10.0


In [7]:
candidates_df.to_csv("all-miniLM-L12-v2_norm-dataset.csv", index=False)

In [8]:
candidates_df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
13461,REJECTED,4.3,4.55,6.50,30,Account Executive,Account Executive,Developer,Healthcare Consultant,Healthcare Consultant,0,2.0,5.0,4.0,8.0
13078,REJECTED,5.2,4.80,5.50,15,Sales Representative,Sales Representative,Graphic designer,Professor,Professor,0,4.0,10.0,3.0,4.5
9978,ACCEPTED,7.0,6.70,8.00,30,Business Development Executive (Inside Sales)|...,Inside Sales Executive,Business development,Sales Manager - Mumbai,Sales Manager,1,2.0,15.0,8.0,14.0
3624,ACCEPTED,8.0,3.00,4.00,30,Chartered Accountant,Chartered Accountant,Finance,senior Analyst-Accounting & Taxation,Tax Accounting Analyst,1,5.0,7.0,3.0,6.0
2199,ACCEPTED,7.0,4.80,6.72,30,Software Associate,Software Associate,Developer,Dot Net Developer,.NET Developer,1,4.0,7.0,6.0,8.5
3022,ACCEPTED,5.0,1.50,2.00,0,Field Executive,Field Executive,Business development,Trainee Associate (Supply/Demand) - Spot - Ban...,Business Trainee,1,0.6,3.0,1.8,2.5
4041,ACCEPTED,5.0,6.00,8.00,15,Career counselor|||Client Partner|||Education ...,Career Counselor,Inside Sales (Outbound),Inside Sales - US,Inside Sales Team,1,1.0,7.0,6.0,9.0
10187,ACCEPTED,6.0,4.65,6.20,1,Territory Sales Leader,Territory Sales Executive,Business development,Area Sales Manager - North,Area Sales Manager,1,3.0,8.0,5.2,7.8
1133,ACCEPTED,5.0,20.00,23.00,30,Assistant Manager Marketing|||Marketing Manager,Assistant Marketing Manager,Digital marketing,Digital Marketing Manager,Digital Marketing Manager,1,3.0,8.0,15.0,24.0
1146,ACCEPTED,5.0,16.00,19.00,15,Brand Marketing manager,Brand Marketing Manager,Digital marketing,Digital Marketing Manager,Digital Marketing Manager,1,3.0,8.0,15.0,24.0
